In [20]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.cluster import KMeans
import lightgbm as lgb
from catboost import CatBoostRegressor, Pool, cv
from sklearn.model_selection import RandomizedSearchCV

In [137]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)

In [138]:
nombres=df.drop(['CompTotal'], axis=1).columns

In [139]:
df.isnull().values.any()

False

In [140]:
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

In [ ]:
limite_inferior = 15000
limite_superior = 80000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1600, 397)


In [142]:
df = df_filtrado

In [143]:
y = df['CompTotal']

In [144]:
df = df.drop(columns=['CompTotal'], axis=1)

In [145]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(df,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [146]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [147]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1280, 396)
(320, 396)
(1280,)
(320,)


In [148]:
best_params0 = {'verbose': 0, 'subsample': 1.0, 'random_seed': 100, 'min_child_samples': 10, 'max_bin': 400, 
                'loss_function': 'RMSE', 'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 500, 
                'grow_policy': 'Lossguide', 'depth': 8, 'colsample_bylevel': 0.9}

In [149]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error


# Definir el modelo con los mejores hiperparámetros
catboost_model = CatBoostRegressor(**best_params0)

# Crear un "scorer" para métricas personalizadas como RMSE
rmse_scorer = make_scorer(mean_squared_error, squared=False)

# Aplicar Cross-Validation
cv_scores = cross_val_score(catboost_model, X_train, y_train, cv=5, scoring=rmse_scorer)

print(f"RMSE promedio: {cv_scores.mean():.2f}")
print(f"Desviación estándar del RMSE: {cv_scores.std():.2f}")

c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in

RMSE promedio: 10042.20
Desviación estándar del RMSE: 501.34


c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# Mejor modelo: 
limite_inferior = 15000
limite_superior = 100000

In [150]:
# Entrenar el modelo 
catboost_model.fit(X_train, y_train)

# Hacer predicciones sobre el conjunto de prueba
y_pred = catboost_model.predict(X_test)

# --- Evaluación del Modelo ---
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.2f}")
print(f"MAPE: {mape:.2f}")

MSE: 93948507.43
RMSE: 9692.70
MAE: 7509.48
R²: 0.64
MAPE: 18.10


# Mismo modelo: 
limite_inferior = 15000
limite_superior = 150000

In [56]:
# # Entrenar el modelo 
# catboost_model.fit(X_train, y_train)

# # Hacer predicciones sobre el conjunto de prueba
# y_pred = catboost_model.predict(X_test)

# # --- Evaluación del Modelo ---
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

# print(f"MSE: {mse:.2f}")
# print(f"RMSE: {rmse:.2f}")
# print(f"MAE: {mae:.2f}")
# print(f"R²: {r2:.2f}")
# print(f"MAPE: {mape:.2f}")

MSE: 210656053.55
RMSE: 14514.00
MAE: 10559.91
R²: 0.64
MAPE: 24.14


Otros parametros

In [114]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)

In [115]:
nombres=df.drop(['CompTotal'], axis=1).columns

In [116]:
df.isnull().values.any()

False

In [117]:
limite_inferior = 15000
limite_superior = 100000

df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

print(f"Tamaño del DataFrame original: {df.shape}")
print(f"Tamaño del DataFrame filtrado: {df_filtrado.shape}")

Tamaño del DataFrame original: (1934, 397)
Tamaño del DataFrame filtrado: (1769, 397)


In [118]:
df = df_filtrado

In [119]:
y = df['CompTotal']

In [120]:
df = df.drop(columns=['CompTotal'], axis=1)

In [121]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(df,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [122]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [123]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1415, 396)
(354, 396)
(1415,)
(354,)


In [127]:
best_params0 = {'verbose': [0], 'subsample': [0.8, 1.0], 'random_seed': [42, 100], 'min_child_samples': [8, 10], 'max_bin': [256, 400], 
                'loss_function': ['RMSE'], 'learning_rate': [0.05, 0.1], 'l2_leaf_reg': [1], 'iterations': [300, 500], 
                'grow_policy': ['Lossguide'], 'depth': [8, 10], 'colsample_bylevel': [0.9]}

In [128]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
# Definir el modelo con los mejores hiperparámetros
catboost_model = CatBoostRegressor()

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
# Crear un "scorer" para métricas personalizadas como RMSE
rmse_scorer = make_scorer(rmse, greater_is_better=False) 


# Crear el objeto RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=catboost_model,
                                   param_distributions=best_params0,
                                   n_iter=200,  # Aumentar el número de iteraciones
                                   scoring=rmse_scorer,
                                   cv=5,
                                   n_jobs=-1)

# Aplicar Cross-Validation
cv_scores = cross_val_score(catboost_model, X_train, y_train, cv=5, scoring=rmse_scorer)

print(f"RMSE promedio: {-cv_scores.mean():.2f}")
print(f"Desviación estándar del RMSE: {cv_scores.std():.2f}")

Learning rate set to 0.041753
0:	learn: 19832.8297693	total: 6.34ms	remaining: 6.33s
1:	learn: 19481.2775093	total: 13.4ms	remaining: 6.7s
2:	learn: 19205.5436653	total: 22.7ms	remaining: 7.56s
3:	learn: 18924.3311453	total: 30.6ms	remaining: 7.63s
4:	learn: 18620.8148377	total: 39.9ms	remaining: 7.94s
5:	learn: 18345.8163256	total: 49.2ms	remaining: 8.15s
6:	learn: 18143.0025916	total: 59.1ms	remaining: 8.38s
7:	learn: 17843.9785704	total: 67.3ms	remaining: 8.34s
8:	learn: 17567.9635300	total: 77.5ms	remaining: 8.54s
9:	learn: 17331.0341896	total: 85.6ms	remaining: 8.47s
10:	learn: 17137.8042765	total: 94.9ms	remaining: 8.53s
11:	learn: 16908.1547992	total: 103ms	remaining: 8.5s
12:	learn: 16678.6309296	total: 111ms	remaining: 8.45s
13:	learn: 16493.9639255	total: 120ms	remaining: 8.47s
14:	learn: 16306.0401323	total: 128ms	remaining: 8.39s
15:	learn: 16093.6305871	total: 136ms	remaining: 8.36s
16:	learn: 15907.1968058	total: 144ms	remaining: 8.3s
17:	learn: 15751.3415303	total: 152ms

In [129]:
# Entrenar el modelo 
random_search.fit(X_train, y_train)
# Resultados
print("Mejores hiperparámetros:", random_search.best_params_)

# --- Evaluación con los mejores hiperparámetros ---
best_model = random_search.best_estimator_

# Hacer predicciones sobre el conjunto de prueba
y_pred = best_model.predict(X_test)

# --- Evaluación del Modelo ---
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.2f}")
print(f"MAPE: {mape:.2f}")

c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 128 is smaller than n_iter=200. Running 128 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Mejores hiperparámetros: {'verbose': 0, 'subsample': 1.0, 'random_seed': 100, 'min_child_samples': 8, 'max_bin': 256, 'loss_function': 'RMSE', 'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 300, 'grow_policy': 'Lossguide', 'depth': 10, 'colsample_bylevel': 0.9}
MSE: 137955651.63
RMSE: 11745.45
MAE: 8934.13
R²: 0.70
MAPE: 20.33
